In [1]:
import pymysql
import pandas as pd
import numpy as np
import datetime
import time
import sys
sys.path.append(r'D:\jupyter files\waiting_time_project\my_tools')
import tools_for_os.for_df as ml_df

In [2]:
sql_re = """SELECT * FROM resource"""
data_re_ = ml_df.get_df_from_sql(sql_re)
print(f'The shape of data_resource {data_re_.shape}')
sql_re_appt = """SELECT * FROM resourceappointment"""
data_re_appt_ = ml_df.get_df_from_sql(sql_re_appt)
print(f'The shape of data_re_appt {data_re_appt_.shape}')
sql_appt = """SELECT * FROM appointment"""
data_appt_ = ml_df.get_df_from_sql(sql_appt)
print(f'The shape of data_appt {data_appt_.shape}')
sql_pl = """SELECT * FROM patientlocation"""
data_pl_ = ml_df.get_df_from_sql(sql_pl)
print(f'The shape of data_pl {data_pl_.shape}')
sql_plmh = """SELECT * FROM patientlocationmh"""
data_plmh_ = ml_df.get_df_from_sql(sql_plmh)
print(f'The shape of data_plmh {data_plmh_.shape}')

The shape of data_resource (477, 6)
The shape of data_re_appt (475920, 7)
The shape of data_appt (1074478, 17)
The shape of data_pl (206855, 9)
The shape of data_plmh (2333039, 10)


In [3]:
try:
    print('Drop columns')
    data_appt_.drop(columns = ['LastUpdated'], inplace = True)
    data_re_.drop(columns = ['LastUpdated'], inplace = True)
    data_re_appt_.drop('LastUpdated', axis = 1, inplace = True)
    data_pl_.drop(columns = ['LastUpdated'], inplace = True)
    data_plmh_.drop(columns = ['LastUpdated'], inplace = True)
except:
    print('Finish droppping columns')

Drop columns


In [4]:
data_re = data_re_.copy()
data_re_appt = data_re_appt_.copy()
data_appt = data_appt_.copy()
data_pl = data_pl_.copy()
data_plmh = data_plmh_.copy()

已经检查过，一个Appointment 里面只有一个AliasSerNum

In [5]:
print(f'The shape of data_appt {data_appt.shape}')
data_appt = data_appt[
    (data_appt.ActualStartDate != datetime.datetime(1970, 1, 1, 0, 0)) &
    (data_appt.ActualEndDate != datetime.datetime(1970, 1, 1, 0, 0)) &
    (data_appt.ActualStartDate != '0000-00-00 00:00:00')]
print(f'The shape of data_appt {data_appt.shape}')

The shape of data_appt (1074478, 16)
The shape of data_appt (370299, 16)


一个AliasSerNum 对应多个ResourceSerNum
一个ResourceSerNum 对应多个AppointmentSerNum

In [6]:
print('Merge data_appt and data_re_appt')
appt_re_appt = pd.merge(data_appt, data_re_appt, on = ['AppointmentSerNum', 'AliasSerNum'], how = 'inner')
print(f'The shape of appt_re_appt {appt_re_appt.shape}')
appt_re_appt.drop('AliasSerNum', axis = 1, inplace = True)

print('\nMerge data_re')
appt_loc = pd.merge(data_re, appt_re_appt, on = ['ResourceSerNum'], how = 'inner')
print(f'The shape of appt_loc {appt_loc.shape}')

print('\nMerge data_pl')
data_pl.drop('AliasSerNum', axis = 1, inplace = True)
appt_loc_pl = pd.merge(data_pl, appt_loc, on = ['AppointmentSerNum'], how = 'right')
print(f'The shape of appt_loc_pl {appt_loc_pl.shape}')

print('\nGet Treat_Date')
appt_loc_pl['Treat_Date'] = appt_loc_pl.apply(lambda x: x.ActualStartDate.strftime("%Y-%m-%d"), axis = 1)

Merge data_appt and data_re_appt
The shape of appt_re_appt (165512, 20)

Merge data_re
The shape of appt_loc (165512, 23)

Merge data_pl
The shape of appt_loc_pl (165512, 29)

Get Treat_Date


In [8]:
appt_loc_pl_grouped = appt_loc_pl.groupby('ResourceSerNum')
appt_loc_pl_grouped.groups.keys()

dict_keys([145, 175, 176, 177, 178, 179, 180, 237, 281, 293, 301, 303, 304, 310, 313, 359, 363, 365, 366, 369, 370, 371, 372, 373, 391, 406, 409, 416, 437, 666, 672, 697])

In [11]:
sample = appt_loc_pl_grouped.get_group(237)
sample.sort_values(by = 'ActualStartDate', inplace = True)
print(sample.shape)

(2930, 30)


C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
sample.columns

Index(['PatientLocationSerNum', 'AppointmentSerNum', 'PatientLocationAriaSer',
       'ResourceSer', 'RevCount', 'CheckedInFlag', 'ArrivalDateTime',
       'ResourceSerNum', 'ResourceAriaSer', 'ResourceName', 'AliasSerNum',
       'ResourceType', 'PatientSerNum', 'AppointmentAriaSer',
       'ActivityInstanceAriaSer', 'DiagnosisSerNum', 'PrioritySerNum',
       'AliasExpressionSerNum', 'Status', 'State', 'ScheduledStartTime',
       'ScheduledEndTime', 'ActualStartDate', 'ActualEndDate',
       'EstimatedDuration', 'HstryUserName', 'ResourceAppointmentSerNum',
       'ExclusiveFlag', 'PrimaryFlag', 'Treat_Date'],
      dtype='object')

In [13]:
sample_grouped = sample.groupby('Treat_Date')
sample_grouped.groups.keys()

dict_keys(['2014-01-30', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19', '2015-03-20', '2015-03-23', '2015-03-24', '2015-03-25', '2015-03-26', '2015-03-27', '2015-03-29', '2015-03-30', '2015-03-31', '2015-04-01', '2015-04-02', '2015-04-07', '2015-04-08', '2015-04-09', '2015-04-10', '2015-04-

In [16]:
sample_grouped.get_group('2015-06-11')

,PatientLocationSerNum,AppointmentSerNum,PatientLocationAriaSer,ResourceSer,RevCount,CheckedInFlag,ArrivalDateTime,ResourceSerNum,ResourceAriaSer,ResourceName,...,ScheduledStartTime,ScheduledEndTime,ActualStartDate,ActualEndDate,EstimatedDuration,HstryUserName,ResourceAppointmentSerNum,ExclusiveFlag,PrimaryFlag,Treat_Date
154578,NaN,641322,NaN,NaN,NaN,NaN,NaT,237,29,CL21EX-B,...,2015-06-11 11:15:00,2015-06-11 11:30:00,2015-06-11 16:00:00,2015-06-11 16:18:00,0.0,vcaissie,36536,1,0,2015-06-11
